In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('/home/pacbio/Heat Map/plaac_candidates_191122.tsv',header=0,sep='\t')

In [3]:
df_name = df["SEQid"][df["SEQid"].str.contains("GN")].str.extract('(?P<OS>.*)OS=(?P<OX>.*)OX=(?P<GN>.*)GN=(?P<PE>.*)PE=(?P<SV>.*)SV=',expand=True)
df_name = df_name.drop("GN",axis=1)

In [4]:
df_name_1 = df["SEQid"][df["SEQid"].str.contains("GN")==False].str.extract('(?P<OS>.*)OS=(?P<OX>.*)OX=(?P<PE>.*)PE=(?P<SV>.*)SV=',expand=True)

In [5]:
df_n = pd.concat([df_name,df_name_1],sort=True)
df_n_1 = df_n.sort_index()

df_nameにタンパク質番号(?)にGNが入ってるやつを改めて処理する。

In [6]:
f1 = lambda x:1 if type(x)==str else 0
a = [i for i, x in enumerate([f1(x) for x in df_n_1["OS"]]) if x == 0]
df_name_2 = df[a[0]:a[0]+1]["SEQid"].str.extract('(?P<OS>.*)OS=(?P<OX>.*)OX=(?P<PE>.*)PE=(?P<SV>.*)SV=',expand=True)
for b in a[1:]:
    df_name_2 = df_name_2.append(df[b:b+1]["SEQid"].str.extract('(?P<OS>.*)OS=(?P<OX>.*)OX=(?P<PE>.*)PE=(?P<SV>.*)SV=',expand=True))
df_n_1 = df_n_1.drop(df_n_1.index[a])
df_n_1 = df_n_1.append(df_name_2)
df_n_1 = df_n_1.sort_index()
df_n_1.to_csv('argonaute_final.tsv', sep='\t', index=True)

ずれてないか確認

In [7]:
f = lambda x,y:1 if x in y else 0
len(df) == sum([f(x,y) for y,x in zip(df["SEQid"],df_n_1["OS"])])

True